# ML Models for TF Features

This notebook uses BOW features to create and evaluate ML models and should therefore be run __after__ ``features_transformer.ipynb``.

### Import packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
import os
warnings.filterwarnings(action="ignore")

from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib
from tqdm.notebook import tqdm
from IPython.display import display_html
import plotly.express as px
import seaborn as sns

from utils import *

Define constants.
- ``PATH``: Path to the base data folder
- ``CPU_CORES``: How many cores to use to process data, default=all - 1
- ``ONLY_SUMMARY``: Whether to use only the summary or the full article
- ``WEIGHTING``: Whether to employ weighting strategy for subdocuments
- ``MEAN``: Whether to calculate the mean of subdocuments
- ``WEIGHT_FUNC``: Which weighting function to use ("tanh", "linear" or "cubic")
- ``MAX_DIST``: Maximum distance for article weights
- ``K_FOLDS``: Number of folds to perform for cross validation
- ``NO_COUNTS``: Remove count based features which are not in the cited papers

In [3]:
PATH = "C:/Users/Tim/.keras/datasets/wikipedia_rss/"
CPU_CORES = os.cpu_count() - 1
ONLY_SUMMARY = True
WEIGHTING = False
MEAN = True
WEIGHT_FUNC = "tanh"
MAX_DIST = 2000  # in m
K_FOLDS = 5
NO_COUNTS = True

Read structured data with added text features.

In [4]:
if ONLY_SUMMARY:
    structured = pd.read_csv(PATH + f"structured_bert_{str(MAX_DIST)}_{WEIGHT_FUNC}.csv")
else:
    structured = pd.read_csv(PATH + f"structured_bert_{str(MAX_DIST)}_{WEIGHT_FUNC}_FULL.csv")
structured = add_dummy_category(structured)

print(structured.shape)
structured.head(10)

(2836, 3094)


,venue_id,latitude,longitude,borough,_category,total_visits,embedding_0,embedding_1,embedding_2,embedding_3,...,dummy_Hookah_Bar,dummy_Hotel_Bar,dummy_Juice_Bar,dummy_Karaoke_Bar,dummy_Piano_Bar,dummy_Pub,dummy_Sake_Bar,dummy_Sports_Bar,dummy_Whisky_Bar,dummy_Wine_Bar
0,3fd66200f964a52001e51ee3,40.726961,-73.980039,Manhattan,Bar,1,-0.062071,-0.097178,-0.069577,0.017937,...,0,0,0,0,0,0,0,0,0,0
1,3fd66200f964a52003e51ee3,40.724822,-73.981456,Manhattan,Bar,15,-0.069718,-0.111770,-0.078443,0.020548,...,0,0,0,0,0,0,0,0,0,0
2,3fd66200f964a52010e51ee3,40.727027,-73.982702,Manhattan,Bar,14,-0.074433,-0.116254,-0.082995,0.021364,...,0,0,0,0,0,0,0,0,0,0
3,3fd66200f964a52011e81ee3,40.762812,-73.967519,Manhattan,Bar,18,-0.094893,-0.144206,-0.103662,0.030769,...,0,0,0,0,0,0,0,0,0,0
4,3fd66200f964a52018e51ee3,40.725112,-73.981278,Manhattan,Bar,29,-0.068102,-0.108868,-0.076608,0.020010,...,0,0,0,0,0,0,0,0,0,0
5,3fd66200f964a5201be41ee3,40.719238,-73.985588,Manhattan,Bar,17,-0.074527,-0.122512,-0.083022,0.023138,...,0,0,0,0,0,0,0,0,0,0
6,3fd66200f964a52025e41ee3,40.724880,-73.994685,Manhattan,Bar,26,-0.111875,-0.174933,-0.120087,0.031510,...,0,0,0,0,0,0,0,0,0,0
7,3fd66200f964a52029e31ee3,40.725638,-73.984561,Manhattan,Bar,15,-0.082956,-0.131200,-0.092229,0.024050,...,0,0,0,0,0,0,0,0,0,0
8,3fd66200f964a5202ee41ee3,40.728543,-73.984699,Manhattan,Bar,34,-0.086315,-0.132255,-0.095396,0.024277,...,0,0,0,0,0,0,0,0,0,0
9,3fd66200f964a52033e61ee3,40.724780,-73.994703,Manhattan,Bar,10,-0.111822,-0.174947,-0.120036,0.031522,...,0,0,0,0,0,0,0,0,0,0


### Recursive Feature Elimination on out-of-sample data

Read data from venues outside of the defined boroughs.

In [5]:
if ONLY_SUMMARY:
    outsample = pd.read_csv(PATH+f"structured_bert_out_borough_{str(MAX_DIST)}_{WEIGHT_FUNC}.csv")
else:
    outsample = pd.read_csv(PATH+f"structured_bert_out_borough_{str(MAX_DIST)}_{WEIGHT_FUNC}_FULL.csv")
outsample = add_dummy_category(outsample)
print(outsample.shape)
outsample.head(10)

(241, 3092)


,venue_id,latitude,longitude,borough,_category,total_visits,embedding_0,embedding_1,embedding_2,embedding_3,...,dummy_Gay_Bar,dummy_Hookah_Bar,dummy_Hotel_Bar,dummy_Juice_Bar,dummy_Karaoke_Bar,dummy_Piano_Bar,dummy_Pub,dummy_Sports_Bar,dummy_Whisky_Bar,dummy_Wine_Bar
0,4b088119f964a520420d23e3,40.891799,-74.211735,NaN,Bar,1,-0.019716,-0.026598,-0.023909,0.003251,...,0,0,0,0,0,0,0,0,0,0
1,4b3e11e8f964a5201d9825e3,40.876140,-74.139872,NaN,Bar,3,-0.016037,-0.020826,-0.019510,0.005818,...,0,0,0,0,0,0,0,0,0,0
2,4b8c7598f964a520dbd132e3,40.871631,-74.087548,NaN,Bar,2,-0.134171,-0.168981,-0.159048,0.026340,...,0,0,0,0,0,0,0,0,0,0
3,4ba24e91f964a520b8eb37e3,40.811090,-74.122510,NaN,Bar,9,-0.103930,-0.152630,-0.148584,0.020659,...,0,0,0,0,0,0,0,0,0,0
4,4bca54b7cc8cd13a108ebdcf,40.662233,-74.264004,NaN,Bar,2,-0.181676,-0.242084,-0.215564,0.030668,...,0,0,0,0,0,0,0,0,0,0
5,4bd0dd5c9854d13aa4a4f84d,40.882928,-74.107832,NaN,Bar,3,-0.157725,-0.203347,-0.164397,0.006368,...,0,0,0,0,0,0,0,0,0,0
6,4c538e72fd2ea59341d49e28,40.858010,-74.147700,NaN,Bar,25,-0.029068,-0.031303,-0.030037,0.002927,...,0,0,0,0,0,0,0,0,0,0
7,4c53b0df479fc9280fb9e391,40.626042,-74.259126,NaN,Bar,12,-0.124913,-0.181569,-0.136162,0.035431,...,0,0,0,0,0,0,0,0,0,0
8,4c55f8bc973fc9289e3137c8,40.876277,-74.123815,NaN,Bar,11,-0.066437,-0.072816,-0.086849,0.009194,...,0,0,0,0,0,0,0,0,0,0
9,4c75c36ab474a1cd8c5fbabf,40.644918,-74.223657,NaN,Bar,1,-0.027942,-0.043120,-0.034990,0.012282,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_refcv = outsample.loc[:, "embedding_0":"dummy_Wine_Bar"]
X_refcv = (X_refcv - X_refcv.mean()) / X_refcv.std()
y_refcv = outsample["total_visits"]

Perform backwards stepwise feature elimination with 5-fold cross validation on out-of-sample data.

In [7]:
# estimator = SVR(kernel="linear")
# selector_text = RFECV(estimator, step=25, cv=5, verbose=0,
#                       n_jobs=-1, scoring='neg_mean_absolute_error')
# selector_text = selector_text.fit(X_refcv, y_refcv)

Save feature selector and create list of worst features according to out-of-sample data.

In [8]:
# _ = joblib.dump(selector_text, f'rfecv_text_{MAX_DIST}.gz')  # save selector
selector_text = joblib.load(f'rfecv_text_{MAX_DIST}.gz')   # load selector

features_sorted = sorted(zip(X_refcv.columns, selector_text.ranking_), key=lambda x: x[1], reverse=True)
worst_features = [x[0] for x in features_sorted if 'embedding_' in x[0]]
worst_features[:5]

['embedding_141',
 'embedding_207',
 'embedding_217',
 'embedding_285',
 'embedding_306']

# Statistical Results

Train Test Split

In [9]:
X = structured.loc[:, "embedding_0":"dummy_Wine_Bar"]
y = structured["total_visits"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

if X.isna().sum().sum() > 0:
    # impute with mean of train
    X_train = X_train.fillna(X_train.mean())
    X_test = X_test.fillna(X_train.mean())

print(f"{X.shape}: {X_train.shape} + {X_test.shape}")
print(f"{y.shape}: {y_train.shape} + {y_test.shape}")

(2836, 3088): (2127, 3088) + (709, 3088)
(2836,): (2127,) + (709,)


### Linear(TF)

In [10]:
for lin_reg in [LinearRegression, Lasso, Ridge]:
    print(f"Regressor: {lin_reg.__name__}")
    reg = make_pipeline(StandardScaler(), lin_reg())
    reg.fit(X_train, y_train)
    lin_pred = reg.predict(X_test)
    metrics_lin = get_metrics(y_test, lin_pred)
    print("")

Regressor: LinearRegression
MAE:  419460.611
RMSE: 2295479.645
NDCG:  0.616

Regressor: Lasso
MAE:  12.626
RMSE: 20.856
NDCG:  0.802

Regressor: Ridge
MAE:  14.963
RMSE: 22.757
NDCG:  0.75



In [11]:
lin_reg = make_pipeline(StandardScaler(), Lasso())
lin_results, lin_cols = cross_validation(lin_reg, X, y, K_FOLDS)

  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  12.79 ± 0.5
RMSE: 20.66 ± 1.53
NDCG: 0.799 ± 0.014


In [12]:
estimator = make_pipeline(StandardScaler(), Lasso())
error_df, col_names, metrics, stds = soos_validation(estimator, structured, return_std=True,
                                                     verbose_drop=False, split_var="borough")
maes, rmses, r_squareds = metrics
list(zip(maes, list(pd.unique(structured["borough"]))))

Predicting borough 1/5
Predicting borough 2/5
Predicting borough 3/5
Predicting borough 4/5
Predicting borough 5/5

Weighted metrics:
MAE:  12.73 ± 2.97
RMSE: 20.89 ± 6.28
NDCG: 0.678 ± 0.14


[(15.2937556950632, 'Manhattan'),
 (9.069085313768253, 'Brooklyn'),
 (8.827842747416152, 'JC + SI'),
 (10.197688722051192, 'Bronx + Queens'),
 (7.694792706579681, 'Newark')]

### SVR(TF)

In [13]:
svr_reg = make_pipeline(StandardScaler(), SVR()).fit(X_train, y_train)
svr_pred = svr_reg.predict(X_test)
metrics_svr = get_metrics(y_test, svr_pred)

MAE:  11.404
RMSE: 22.586
NDCG:  0.784


In [14]:
svr_results, svr_cols = cross_validation(svr_reg, X, y, K_FOLDS)

  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  11.36 ± 0.61
RMSE: 21.97 ± 1.64
NDCG: 0.812 ± 0.009


In [15]:
estimator = make_pipeline(StandardScaler(), SVR())
error_df, col_names, metrics = soos_validation(estimator, structured,
                                               verbose_drop=False, split_var="borough")
maes, rmses, r_squareds = metrics

Predicting borough 1/5
Predicting borough 2/5
Predicting borough 3/5
Predicting borough 4/5
Predicting borough 5/5

Weighted metrics:
MAE:  11.63 ± 4.35
RMSE: 21.9 ± 7.25
NDCG: 0.747 ± 0.1


## Feature Selection

Use worst features from out-of-sample data to drop features and observe change in SOOS validation metrics.

In [16]:
# results = []
# for i_worst in tqdm(range(0, 3051, 25)):
#     estimator = make_pipeline(StandardScaler(), SVR())
#     structured_reduced = structured.drop(worst_features[:i_worst], axis=1)
#     error_df, col_names, metrics = soos_validation(estimator, structured_reduced, verbose=False,
#                                                    verbose_drop=False, split_var="borough")
#     maes, rmses, ndcg = metrics
#     results.append(metrics)
    
# results_df = pd.DataFrame(np.array(results), columns=["MAE", "RMSE", "NDCG"], index=feature_count)
# results_df.to_csv(PATH+"results/metrics_soos_bert_reduced.csv")

In [17]:
results_df = pd.read_csv(PATH+"results/metrics_soos_bert_reduced.csv", index_col=0)
print(results_df.shape)
results_df.head()

(123, 3)


,MAE,RMSE,NDCG
0,11.632802,21.899323,0.746524
25,11.633564,21.901650,0.747230
50,11.634180,21.903752,0.747047
75,11.635130,21.905961,0.748132
100,11.635836,21.907948,0.748710


Retrain model with lowest MAE score and view random cross validation as well as SOOS validation metrics.

In [18]:
mae_lst = list(results_df["MAE"])
lowest_idx = np.argmin(mae_lst)
feature_count = list(range(0, 3051, 25))
best_feature_count = feature_count[lowest_idx]
print(f"Lowest MAE with {structured.shape[1] - best_feature_count}/{structured.shape[1]} features\n")

estimator = make_pipeline(StandardScaler(), SVR())
structured_reduced = structured.drop(worst_features[:best_feature_count], axis=1)

print(f"{K_FOLDS}-fold random cross validation")
X = structured_reduced.loc[:, "total_visits":"dummy_Wine_Bar"].drop(["total_visits"], axis=1)
svr_results, svr_cols = cross_validation(estimator, X, y, K_FOLDS)

print("\nspatial out-of-sample validation\n")
error_df, col_names, metrics = soos_validation(estimator, structured_reduced,
                                               verbose_drop=False, split_var="borough")
maes, rmses, ndcg = metrics

error_df.to_csv(PATH+"results/bert_error_df.csv", index=False)

Lowest MAE with 94/3094 features

5-fold random cross validation


  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  11.05 ± 0.62
RMSE: 21.54 ± 1.65
NDCG: 0.831 ± 0.008

spatial out-of-sample validation

Predicting borough 1/5
Predicting borough 2/5
Predicting borough 3/5
Predicting borough 4/5
Predicting borough 5/5

Weighted metrics:
MAE:  11.56 ± 4.33
RMSE: 21.9 ± 7.3
NDCG: 0.754 ± 0.103


MAE split up for each borough.

In [19]:
error_df, col_names, metrics, stds = soos_validation(estimator, structured_reduced, verbose=False,
                                                     verbose_drop=False, split_var="borough",
                                                     return_std=True)
maes, rmses, ndcg = metrics
list(zip(maes, list(pd.unique(structured_reduced["borough"]))))

[(15.186984924319576, 'Manhattan'),
 (4.824377563852768, 'Brooklyn'),
 (6.996361103896562, 'JC + SI'),
 (8.794011281902563, 'Bronx + Queens'),
 (4.295761728717104, 'Newark')]

## Combining Text and Geo Features

Read geo features.

In [20]:
geo_features = pd.read_csv(PATH + f"structured_geo_features.csv")
geo_features = geo_features.dropna(axis=0)

print(geo_features.shape)
geo_features.head(10)

(2836, 434)


,venue_id,latitude,longitude,borough,category,org_category,total_visits,jazz_club_count,gym_count,indian_restaurant_count,...,cricket_ground_count,campaign_office_count,rock_climbing_spot_count,yogurt_count,volcano_count,area_density,area_entropy,competition,traffic_access,dist_center
0,3fd66200f964a52001e51ee3,40.726961,-73.980039,Manhattan,Bar,Dive Bar,1,0,0,0,...,0,0,0,0,0,79,5.301005,-0.101266,-2.349061,0.044945
1,3fd66200f964a52003e51ee3,40.724822,-73.981456,Manhattan,Bar,Dive Bar,15,0,1,0,...,0,0,0,0,0,102,5.620873,-0.049020,-2.443636,0.046575
2,3fd66200f964a52010e51ee3,40.727027,-73.982702,Manhattan,Bar,Dive Bar,14,0,0,0,...,0,0,0,0,0,184,5.954679,-0.076087,-2.051850,0.047595
3,3fd66200f964a52011e81ee3,40.762812,-73.967519,Manhattan,Bar,Dive Bar,18,0,3,1,...,0,0,0,0,0,229,6.029996,-0.000000,-1.776750,0.045594
4,3fd66200f964a52018e51ee3,40.725112,-73.981278,Manhattan,Bar,Dive Bar,29,0,1,0,...,0,0,0,0,0,107,5.547164,-0.065421,-2.432629,0.046363
5,3fd66200f964a5201be41ee3,40.719238,-73.985588,Manhattan,Bar,Dive Bar,17,1,0,0,...,0,0,0,0,0,148,6.037900,-0.087838,-2.242368,0.051614
6,3fd66200f964a52025e41ee3,40.724880,-73.994685,Manhattan,Bar,Dive Bar,26,1,2,2,...,0,0,0,0,0,302,5.952141,-0.009934,-2.154852,0.059719
7,3fd66200f964a52029e31ee3,40.725638,-73.984561,Manhattan,Bar,Dive Bar,15,0,0,8,...,0,0,0,0,0,204,6.107098,-0.088235,-2.386347,0.049569
8,3fd66200f964a5202ee41ee3,40.728543,-73.984699,Manhattan,Bar,Dive Bar,34,0,0,1,...,0,0,0,0,0,252,6.164078,-0.067460,-2.117217,0.049500
9,3fd66200f964a52033e61ee3,40.724780,-73.994703,Manhattan,Bar,Dive Bar,10,1,2,2,...,0,0,0,0,0,307,5.964675,-0.009772,-2.174384,0.059746


In [21]:
if NO_COUNTS:
    X_geo = geo_features.loc[:, "area_density":"dist_center"]
else:
    X_geo = geo_features.loc[:, "jazz_club_count":"dist_center"]
    
X_combined = pd.concat([X, X_geo], axis=1)
structured_combined = pd.concat([structured_reduced, X_geo], axis=1)

Train Test split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.25, random_state=42)

if X_combined.isna().sum().sum() > 0:
    # impute with mean of train
    X_train = X_train.fillna(X_train.mean())
    X_test = X_test.fillna(X_train.mean())

print(f"{X_combined.shape}: {X_train.shape} + {X_test.shape}")
print(f"{y.shape}: {y_train.shape} + {y_test.shape}")

(2836, 93): (2127, 93) + (709, 93)
(2836,): (2127,) + (709,)


### Linear(GEO+TF)

In [23]:
for lin_reg in [LinearRegression, Lasso, Ridge]:
    print(f"Regressor: {lin_reg.__name__}")
    reg = make_pipeline(StandardScaler(), lin_reg())
    reg.fit(X_train, y_train)
    lin_pred = reg.predict(X_test)
    metrics_lin = get_metrics(y_test, lin_pred)
    print("")

Regressor: LinearRegression
MAE:  12.762
RMSE: 20.448
NDCG:  0.802

Regressor: Lasso
MAE:  12.586
RMSE: 20.74
NDCG:  0.802

Regressor: Ridge
MAE:  12.749
RMSE: 20.438
NDCG:  0.801



In [24]:
lin_reg = make_pipeline(StandardScaler(), Lasso())
lin_results, lin_cols = cross_validation(lin_reg, X_combined, y, K_FOLDS)

  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  12.64 ± 0.51
RMSE: 20.52 ± 1.53
NDCG: 0.806 ± 0.015


In [25]:
estimator = make_pipeline(StandardScaler(), Lasso())
error_df, col_names, metrics, stds = soos_validation(estimator, structured_combined, return_std=True,
                                                     verbose_drop=False, split_var="borough")
maes, rmses, ndcg = metrics
list(zip(maes, list(pd.unique(structured_reduced["borough"]))))

Predicting borough 1/5
Predicting borough 2/5
Predicting borough 3/5
Predicting borough 4/5
Predicting borough 5/5

Weighted metrics:
MAE:  12.77 ± 3.66
RMSE: 20.22 ± 6.23
NDCG: 0.708 ± 0.131


[(15.848864281771224, 'Manhattan'),
 (7.25950617705396, 'Brooklyn'),
 (8.379148303021296, 'JC + SI'),
 (10.425916266498389, 'Bronx + Queens'),
 (6.784667647977543, 'Newark')]

### SVR(GEO+TF)

In [26]:
svr_reg = make_pipeline(StandardScaler(), SVR()).fit(X_train, y_train)
svr_pred = svr_reg.predict(X_test)
metrics_svr = get_metrics(y_test, svr_pred)

MAE:  11.085
RMSE: 22.088
NDCG:  0.813


In [27]:
svr_results, svr_cols = cross_validation(svr_reg, X_combined, y, K_FOLDS)

  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  11.03 ± 0.63
RMSE: 21.52 ± 1.65
NDCG: 0.832 ± 0.009


In [28]:
estimator = make_pipeline(StandardScaler(), SVR())
error_df, col_names, metrics, stds = soos_validation(estimator, structured_combined, return_std=True,
                                               verbose_drop=False, split_var="borough")
maes, rmses, ndcg = metrics
list(zip(maes, list(pd.unique(structured_reduced["borough"]))))

Predicting borough 1/5
Predicting borough 2/5
Predicting borough 3/5
Predicting borough 4/5
Predicting borough 5/5

Weighted metrics:
MAE:  11.54 ± 4.36
RMSE: 21.86 ± 7.33
NDCG: 0.747 ± 0.117


[(15.193971473784865, 'Manhattan'),
 (4.761008863892915, 'Brooklyn'),
 (6.939707715847376, 'JC + SI'),
 (8.746625381233661, 'Bronx + Queens'),
 (4.182764009386757, 'Newark')]